## ***Installing dependencies***



In [1]:
pip install simpletransformers "transformers==4.30.2"

In [2]:
#dependencies
!pip install flask
!pip install pyngrok

In [3]:
#installing dependencies
!pip install ngrok

In [4]:
pip install sentencepiece sacremoses

## ***Importing the transformer and loading the tokenizer***



In [5]:
import transformers

In [6]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizerFast
import pandas as pd
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")


## ***Implementing the paraphrasing function***



In [7]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

## ***Example***



In [8]:
sentence = "i spent $3000 for the printing and advertising material distribution service."
test=get_paraphrased_sentences(model, tokenizer, sentence, num_beams=10, num_return_sequences=1)
test

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['The printing and advertising material distribution service cost me $3000.']

## ***Flask Endpoint***



In [9]:
from flask import Flask, request, jsonify
from transformers import PegasusForConditionalGeneration, PegasusTokenizerFast
from pyngrok import ngrok

# Create a Flask app
app = Flask(__name__)
ngrok.set_auth_token("YOUR_AUTH_TOKEN_HERE")
public_url = ngrok.connect(5000).public_url

# Load the machine learning model
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
    inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
    outputs = model.generate(
        **inputs,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

@app.route('/paraphrase', methods=['POST'])
def paraphrase():
    data = request.get_json()
    input_text = data.get('text')

    if input_text:
        paraphrased_sentences = get_paraphrased_sentences(model, tokenizer, input_text, num_beams=10, num_return_sequences=1)
        return jsonify({'paraphrased_text': paraphrased_sentences[0]})
    else:
        return jsonify({'error': 'Text not provided'}), 400

# Start the Flask app
if __name__ == "__main__":
    print(f"To access the global link, please click {public_url}")
    app.run(debug=False)


To access the global link, please click https://0df5-34-90-147-253.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [09/Aug/2023 22:04:51] "POST /paraphrase HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Aug/2023 22:05:07] "POST /paraphrase HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Aug/2023 22:05:44] "POST /paraphrase HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Aug/2023 22:06:41] "POST /paraphrase HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Aug/2023 22:07:31] "POST /paraphrase HTTP/1.1" 200 -
INFO:werkzeug:127.0